<a href="https://colab.research.google.com/github/AI4Bharat/indicTrans/blob/main/indictrans_fairseq_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# create a seperate folder to store everything
!mkdir testing
%cd testing

/content/testing


In [ ]:
# clone the repo for running evaluation
!git clone https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/rsennrich/subword-nmt.git
%cd ..

Cloning into 'indicTrans'...
remote: Enumerating objects: 398, done.
remote: Counting objects: 100% (398/398), done.
remote: Compressing objects: 100% (267/267), done.
remote: Total 398 (delta 231), reused 251 (delta 126), pack-reused 0
Receiving objects: 100% (398/398), 1.41 MiB | 6.82 MiB/s, done.
Resolving deltas: 100% (231/231), done.
/content/testing/indicTrans
Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1325, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 1325 (delta 84), reused 89 (delta 41), pack-reused 1178
Receiving objects: 100% (1325/1325), 9.57 MiB | 7.40 MiB/s, done.
Resolving deltas: 100% (688/688), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 133 (delta 0), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (133/133), 149.77 MiB

In [ ]:
# Install the necessary libraries
!pip install sacremoses pandas mock sacrebleu tensorboardX pyarrow indic-nlp-library
# Install fairseq from source
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
# !git checkout da9eaba12d82b9bfc1442f0e2c6fc1b895f4d35d
!pip install ./
! pip install xformers
%cd ..

     |████████████████████████████████| 901kB 3.9MB/s 
     |████████████████████████████████| 61kB 8.3MB/s 
     |████████████████████████████████| 122kB 35.5MB/s 
     |████████████████████████████████| 40kB 5.8MB/s 
     |████████████████████████████████| 9.2MB 28.0MB/s 
     |████████████████████████████████| 552kB 30.6MB/s 
  Created wheel for sphinx-argparse: filename=sphinx_argparse-0.2.5-cp37-none-any.whl size=11552 sha256=d8804d903bcf829240052e806acb7c6051e0c240bddf22ef8bd4e4bd2abdfbac
  Stored in directory: /root/.cache/pip/wheels/2a/18/1b/4990a1859da4edc77ab312bc2986c08d2733fb5713d06e44f5
Successfully built sphinx-argparse
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: docutils 0.17.1
    Uninstalling docutils-0.17.1:
      Successfully uninstalled docutils-0.17.1
Cloning into 'fairseq'...
remote: Enumerating objects: 28243, done.
remote: Counting objects: 100% (62/62), done.
remote:

In [ ]:
# add fairseq folder to python path
import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"
# sanity check to see if fairseq is installed
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils

In [ ]:
# download the indictrans model


# downloading the indic-en model
!wget https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/indic2en.zip
!unzip indic2en.zip

# downloading the en-indic model
# !wget https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/en2indic.zip
# !unzip en2indic.zip

# # downloading the indic-indic model
# !wget https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/m2m.zip
# !unzip m2m.zip

%cd indicTrans

--2021-06-09 15:06:00--  https://storage.googleapis.com/samanantar-public/V0.2/models/indic-en.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.188.128, 64.233.189.128, 108.177.97.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.188.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4551079075 (4.2G) [application/zip]
Saving to: ‘indic-en.zip’

indic-en.zip        100%[===================>]   4.24G  49.9MB/s    in 1m 47s  

2021-06-09 15:07:48 (40.5 MB/s) - ‘indic-en.zip’ saved [4551079075/4551079075]

Archive:  indic-en.zip
   creating: indic-en/
   creating: indic-en/vocab/
  inflating: indic-en/vocab/bpe_codes.32k.SRC  
  inflating: indic-en/vocab/vocab.SRC  
  inflating: indic-en/vocab/vocab.TGT  
  inflating: indic-en/vocab/bpe_codes.32k.TGT  
   creating: indic-en/final_bin/
  inflating: indic-en/final_bin/dict.TGT.txt  
  inflating: indic-en/final_bin/dict.SRC.txt  
   creating: indic-en/model/
  infl

In [ ]:
# creating a text file and adding en sentences we can use for testing the model
!touch en_sentences.txt
!echo 'This bicycle is too small for you !!' >> en_sentences.txt
!echo "I will directly meet you at the airport." >> en_sentences.txt
!echo 'If COVID-19 is spreading in your community, stay safe by taking some simple precautions, such as physical distancing, wearing a mask, keeping rooms well ventilated, avoiding crowds, cleaning your hands, and coughing into a bent elbow or tissue' >> en_sentences.txt

In [ ]:
# joint_translate takes src_file, output_fname, src_lang, tgt_lang, model_folder as inputs
# src_file -> input text file to be translated
# output_fname -> name of the output file (will get created) containing the model predictions
# src_lang -> source lang code of the input text ( in this case we are using en-indic model and hence src_lang would be 'en')
# tgt_lang -> target lang code of the input text ( tgt lang for en-indic model would be any of the 11 indic langs we trained on:
#              as, bn, hi, gu, kn, ml, mr, or, pa, ta, te)
# supported languages are:
#              as - assamese, bn - bengali, gu - gujarathi, hi - hindi, kn - kannada, 
#              ml - malayalam, mr - marathi, or - oriya, pa - punjabi, ta - tamil, te - telugu

# model_dir -> the directory containing the model and the vocab files

# Note: if the translation is taking a lot of time, please tune the buffer_size and batch_size parameter for fairseq-interactive defined inside this joint_translate script


# here we are translating the english sentences to tamil
!bash joint_translate.sh en_sentences.txt ta_outputs.txt 'en' 'ta' '../en-indic'

Wed Jun 9 15:18:01 UTC 2021
Applying normalization and script conversion
100% 3/3 [00:00<00:00, 71.78it/s]
Number of sentences in input: 3
Applying BPE
Decoding
Extracting translations, script conversion and detokenization
Translation completed


In [ ]:
!cat ta_outputs.txt

இந்த சைக்கிள் உங்களுக்கு மிகவும் சிறியது!
விமான நிலையத்தில் உங்களை நேரில் சந்திக்கிறேன்.
உங்கள் சமூகத்தில் கோவிட்-19 பரவுகிறது என்றால், சில எளிய முன்னெச்சரிக்கை நடவடிக்கைகளான, தனி நபர் இடைவெளி, முகக்கவசம் அணிதல், அறைகளை நன்கு காற்றோட்டமாக வைத்திருத்தல், கூட்டத்தைத் தவிர்த்தல், கைகளைக் கழுவுதல், முழங்கை அல்லது திசுக்களில் இருமல் போன்றவற்றை மேற்கொள்வதன் மூலம் பாதுகாப்பாக இருங்கள்.


In [ ]:
# Similarly, we can translate the english sentences to hindi
!bash joint_translate.sh en_sentences.txt hi_outputs.txt 'en' 'hi' '../en-indic'

Wed Jun 9 15:21:31 UTC 2021
Applying normalization and script conversion
100% 3/3 [00:00<00:00, 88.59it/s]
Number of sentences in input: 3
Applying BPE
Decoding
Extracting translations, script conversion and detokenization
Translation completed


In [ ]:
!cat hi_outputs.txt

यह साइकिल तुम्हारे लिए बहुत छोटी है!
मैं आपसे एयरपोर्ट पर ही मिलने वाला हूं।
यदि आपके समुदाय में कोविड-19 फैल रहा है, तो कुछ सरल सावधानियां बरतें, जैसे शारीरिक दूरी बनाए रखना, मास्क पहनना, कमरों को अच्छी तरह से हवादार रखना, भीड़ से बचना, अपने हाथों को साफ करना और कोहनी या ऊतक को मोड़कर खांसते हुए सुरक्षित रहें


In [ ]:
# creating a text file and adding hi sentences we can use for testing the model
!touch hi_sentences.txt
!echo 'तुम आज सुबह यहाँ क्यों आए?' >> hi_sentences.txt
!echo "मेरे परिवार में हर कोई जल्दी उठता है।" >> hi_sentences.txt
!echo ' स्वास्थ्य और परिवार कल्याण मंत्रालय द्वारा प्रदान की गई जानकारी और सलाह को सावधानी व सही तरीके से पालन कर वायरस के स्थानीय प्रसार को रोका जा सकता है।' >> hi_sentences.txt

!touch ta_sentences.txt
!echo 'அவனுக்கு நம்மைப் தெரியும் என்று தோன்றுகிறது' >> ta_sentences.txt
!echo "இது எங்கே இருக்கு என்று என்னால் கண்டுபிடிக்க முடியவில்லை." >> ta_sentences.txt
!echo 'உங்களுக்கு உங்கள் அருகில் இருக்கும் ஒருவருக்கோ இத்தகைய அறிகுறிகள் தென்பட்டால், வீட்டிலேயே இருப்பது, கொரோனா வைரஸ் தொற்று பிறருக்கு வராமல் தடுக்க உதவும்.' >> ta_sentences.txt

In [ ]:
# here we are translating the english sentences to hindi
!bash joint_translate.sh hi_sentences.txt en_outputs.txt 'hi' 'en' '../indic-en'

Wed Jun 9 15:24:43 UTC 2021
Applying normalization and script conversion
100% 3/3 [00:00<00:00, 74.90it/s]
Number of sentences in input: 3
Applying BPE
Decoding
Extracting translations, script conversion and detokenization
Translation completed


In [ ]:
! cat en_outputs.txt

Why did you come here this morning?
Everyone in my family gets up early.
The local spread of the virus can be curbed by following the information and advice provided by the Ministry of Health and Family Welfare in a careful and correct manner.


In [ ]:
# here we are translating the english sentences to tamil
!bash joint_translate.sh ta_sentences.txt en_outputs.txt 'ta' 'en' '../indic-en'

Wed Jun 9 15:28:05 UTC 2021
Applying normalization and script conversion
100% 3/3 [00:00<00:00, 72.92it/s]
Number of sentences in input: 3
Applying BPE
Decoding
Extracting translations, script conversion and detokenization
Translation completed


In [ ]:
! cat en_outputs.txt

He seems to know us.
I couldnt find it anywhere.
If someone in your neighbourhood develops these symptoms, staying at home can help prevent the spread of the coronavirus infection.


In [ ]:
# we just rename the m2m_joint_vocab file here as joint_translate uses bpe_codes.32k.SRC
mv ../m2m/vocab/bpe_codes.32k.SRC_TGT ../m2m/vocab/bpe_codes.32k.SRC

In [ ]:
# here we are using the indic2indic model for translating the hindi sentences to tamil
!bash joint_translate.sh hi_sentences.txt ta_outputs.txt 'hi' 'ta' '../m2m'

Wed Jun 9 15:39:26 UTC 2021
Applying normalization and script conversion
100% 3/3 [00:00<00:00, 63.53it/s]
Number of sentences in input: 3
Applying BPE
Decoding
Extracting translations, script conversion and detokenization
Translation completed


In [ ]:
 ! cat hi_sentences.txt # the hindi inputs

तुम आज सुबह यहाँ क्यों आए?
मेरे परिवार में हर कोई जल्दी उठता है।
 स्वास्थ्य और परिवार कल्याण मंत्रालय द्वारा प्रदान की गई जानकारी और सलाह को सावधानी व सही तरीके से पालन कर वायरस के स्थानीय प्रसार को रोका जा सकता है।


In [ ]:
! cat ta_outputs.txt # the tamil outputs

ஏன் இன்று காலையில் வந்தீர்கள்?
எனது குடும்பத்தில் உள்ள ஒவ்வொருவரும் விரைவில் எழுவார்கள்.
மத்திய சுகாதாரம் மற்றும் குடும்ப நல அமைச்சகத்தின் அறிவுறுத்தல்கள் மற்றும் தகவல்களைப் பின்பற்றுவதன் மூலம், உள்ளூர் அளவில் வைரஸ் பரவுவதைத் தடுக்க முடியும்.


In [ ]:
# here we are using the indic2indic model for translating the hindi sentences to tamil (same as above with reversing the direction)
!bash joint_translate.sh ta_sentences.txt hi_outputs.txt 'ta' 'hi' '../m2m'

Wed Jun 9 15:45:53 UTC 2021
Applying normalization and script conversion
100% 3/3 [00:00<00:00, 82.25it/s]
Number of sentences in input: 3
Applying BPE
Decoding
Extracting translations, script conversion and detokenization
Translation completed


In [ ]:
! cat ta_sentences.txt # the tamil inputs

அவனுக்கு நம்மைப் தெரியும் என்று தோன்றுகிறது
இது எங்கே இருக்கு என்று என்னால் கண்டுபிடிக்க முடியவில்லை.
உங்களுக்கு உங்கள் அருகில் இருக்கும் ஒருவருக்கோ இத்தகைய அறிகுறிகள் தென்பட்டால், வீட்டிலேயே இருப்பது, கொரோனா வைரஸ் தொற்று பிறருக்கு வராமல் தடுக்க உதவும்.


In [ ]:
! cat hi_outputs.txt   # the hi outputs

ऐसा लगता है कि वह हमें जानता है।
मुझे पता नहीं था कि यह कहां है।
अगर आपके आस-पास के किसी व्यक्ति में ऐसे लक्षण दिखाई देते हैं, तो घर पर रहने से कोरोना वायरस को फैलने से रोकने में मदद मिलेगी।


In [ ]:
# to compute bleu scores for the predicitions with a reference file, use the following command

# bash compute_bleu.sh pred_fname ref_fname src_lang tgt_lang
# arguments:
# pred_fname: file that contains model predictions
# ref_fname: file that contains references
# src_lang and tgt_lang : the source and target language